Complete the exercises below For **Assignment #13**.

Load the `ISLR2` and the `tidymodels` packages.

In [60]:
library(ISLR2)
library(tidymodels)

In this assignment we will use the `Default` dataset which includes the default status for credit card customers (`default` variable) in addition to each customer's:

1. credit card balance (`balance` variable),
1. student status (`student` variable), and,
1. income (`income` variable).

In [61]:
Default |> head()

,default,student,balance,income
,<fct>,<fct>,<dbl>,<dbl>
1,No,No,729.5265,44361.625
2,No,Yes,817.1804,12106.135
3,No,No,1073.5492,31767.139
4,No,No,529.2506,35704.494
5,No,No,785.6559,38463.496
6,No,Yes,919.5885,7491.559


We will be modeling `default` with the customer features.

Before we begin let's count how many customers fall into each `default` category.

In [62]:
Default |> count(default)

default,n
<fct>,<int>
No,9667
Yes,333


The data is quite imbalanced. This will be important to keep in mind when we evaluate the performance of our model later. 

Run the code below to create and training data from `Default`. We will use the "test" dataset at the end to get a final evaluation of our best model's accuracy.

In [63]:
Default_split = initial_split(Default, prop = 0.90, strata = default)
Default_train = training(Default_split)
Default_test = testing(Default_split)

Create a logistic regression model called `mod`. Set the engine to `glm` and the mode to `classification`. 

In [64]:
mod = logistic_reg() |>
  set_engine("glm") |>
  set_mode("classification")

Our data is imbalanced. As such, a naive model that *always* predicts a customer to **not default** would be correct quite often. Let's start by calculating the "accuracy" of a naive model. This will be the baseline accuracy by which we evaluate other models.

In [65]:
# This code calculates the accuracy of a model that always predicts default to be "No"
NM = Default_train |>
    mutate(.pred_naive = factor('No', levels = c('No', 'Yes')))
NM |> head()
    NM |> accuracy(truth = default, .pred_naive)

,default,student,balance,income,.pred_naive
,<fct>,<fct>,<dbl>,<dbl>,<fct>
1,No,No,729.5265,44361.625,No
2,No,No,1073.5492,31767.139,No
3,No,No,529.2506,35704.494,No
4,No,No,785.6559,38463.496,No
5,No,Yes,919.5885,7491.559,No
6,No,No,825.5133,24905.227,No


.metric,.estimator,.estimate
<chr>,<chr>,<dbl>
accuracy,binary,0.9667778


Let's use k-fold cross validation to evaluate the performance of a model where the outcome is `default` and the predictors are `income` and `balance`.

To start, use `vfold_cv` to generate 10 validation folds (i.e. set the `v` variable to 10). Set the `strata` argument to `default` so we preserve the distribution of `default` values in each fold.

Creat your folds below and use `glimpse` to look at the output table. Call your output folds tables "folds".

In [66]:
set.seed(1)
folds = vfold_cv(Default_train, v = 10, strata = default)
glimpse(folds)

Rows: 10
Columns: 2
$ splits <list> [<vfold_split[8100 x 900 x 9000 x 4]>], [<vfold_split[8100 x 9…
$ id     <chr> "Fold01", "Fold02", "Fold03", "Fold04", "Fold05", "Fold06", "Fo…


The code below fits a model to each of your 10 folds. `collect_metrics` finds the average of evaluation metrics for each of your ten models. 

In [67]:
set.seed(7)
fit = mod |> fit_resamples(default ~ income + balance, folds)
fit |> collect_metrics()

.metric,.estimator,mean,n,std_err,.config
<chr>,<chr>,<dbl>,<int>,<dbl>,<chr>
accuracy,binary,0.9734444,10,0.001329728,Preprocessor1_Model1
roc_auc,binary,0.9486501,10,0.003765292,Preprocessor1_Model1


❓How does the model accuracy compare to the naive model from above?

* **1st model - The accuracy of the naive model was approximately 0.967.**
* **2nd model - The accuracy model that uses predictors as income and balance is approximately 0.973.**
* **The accuracy of the 2nd model is slightly higher than the first model. The second model performs slightly better than the first.**

Complete the cell below to evaluate a model also includes the `student` variable as as predictor.
1. use `default ~ income + balance + student` as the formula,
2. encode your `student` variable with `step_dummy`, and,
3. don't forget to `prep` your recipe!

In [68]:
rec = recipe(default ~ income + balance + student, data = Default_train) |>
  step_dummy(student) |>
  prep()
mod |> fit_resamples(rec, folds) |>
collect_metrics()

.metric,.estimator,mean,n,std_err,.config
<chr>,<chr>,<dbl>,<int>,<dbl>,<chr>
accuracy,binary,0.9727778,10,0.001368367,Preprocessor1_Model1
roc_auc,binary,0.9487186,10,0.003893002,Preprocessor1_Model1


❓Does it appear that the model that includes `student` improves upon the first model with only `income` and `balance` as predictors?

**No, the model with student does not improve upon the model with only income and balance. The accuracy for the model with a student is slightly lesser(0.9727778)**

Finally, estimate the accuracy of an `default ~ income + balance` model on the test data, `Default_test`. 

❓Does our model outperform a naive model?

In [69]:
rec_test <- recipe(default ~ income + balance, data = Default_test) |>
  prep()
wf <- workflow() |>
  add_recipe(rec) |>
  add_model(mod)

set.seed(3)
fit_test <- fit(wf, data = Default_test)

pred <- predict(fit_test, new_data = Default_test) |>
  bind_cols(predict(fit_test, Default_test, type = "prob")) |>
  bind_cols(Default_test |> select(default))
pred |> head()
pred |> accuracy(truth = default, .pred_class)
pred |> roc_auc(truth = default, .pred_No)

.pred_class,.pred_No,.pred_Yes,default
<fct>,<dbl>,<dbl>,<fct>
No,0.9994043,0.0005957331,No
No,0.9985073,0.0014926919,No
No,0.9992997,0.0007003459,No
No,0.9975306,0.0024694239,No
No,0.9543961,0.0456038751,No
No,0.8900379,0.1099621261,No


.metric,.estimator,.estimate
<chr>,<chr>,<dbl>
accuracy,binary,0.978


.metric,.estimator,.estimate
<chr>,<chr>,<dbl>
roc_auc,binary,0.9528072


**The accuracy of the test data model is higher by 0.0112(1.12%), so we can say it outperforms the naive model. I do not know what the differnce needs to be before it can be qualified as an outperformance. Just any difference?**

In [59]:
# Default_train |> count(default)
# Default_test |> count(default)

**I need to spend some more time studying this, I am not able to provide as much conclusion as I will like about my results**